<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/Train_and_check_IG_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    base='Google Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.utils.data
import torch.nn as nn
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")
import random
import pickle as pkl
import pandas as pd
from scipy.stats import entropy


In [13]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available()

True

In [14]:
def decimal_to_binary(integer, n_hid):
    string=bin(int(integer))[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def Ising_energy(v_list):
    L = 3
    E_list=[]
    for n in range(len(v_list)):
        v=v_list[n]
        E = 0
        for i in range(L):
            for j in range(L):
                s = v[i,j]
                neigh = v[(i+1)%L, j] + v[i,(j+1)%L] + v[(i-1)%L,j] + v[i,(j-1)%L] 
                E += -neigh * s
        E_list.append(E/2)
    return np.array(E_list)

# Define RBM graph

In [15]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        self.v_bias = nn.Parameter(torch.ones(1, n_vis).to(device))
        self.h_bias = nn.Parameter(torch.zeros(1, n_hid).to(device))
        self.Weight = nn.Parameter(torch.randn(n_hid, n_vis).to(device))
        self.k = k


    def v2h(self, v):
        return torch.sigmoid(F.linear(v, self.Weight, self.h_bias))

    def h2v(self, h):
        return torch.sigmoid(F.linear(h, self.Weight.t(), self.v_bias))
    
    def Fv(self, v):
        v_term = torch.matmul(v, self.v_bias.t()).view(len(v))
        h_term = torch.sum(F.softplus(F.linear(v, self.Weight, self.h_bias)), dim=1)
        return -h_term -v_term

    def energy(self, v, h):
        v=v.bernoulli()
        h=h.bernoulli()
        return -torch.matmul(v, self.v_bias.t())-torch.matmul(torch.matmul(v, self.Weight.t()),h.t())-torch.matmul(h, self.h_bias.t())
    
    def Energy_GPU2(self, v_list0, h_list0):
        if CUDA:
            n_split=torch.cuda.device_count()
        else:
            n_split=1
        e_list=[]
        m_split=1
        for j in range(m_split):
            v_list1=torch.stack(list(v_list0[j*int(len(v_list0)/m_split):(j+1)*int(len(v_list0)/m_split)]))
            h_list1=torch.stack(list(h_list0[j*int(len(h_list0)/m_split):(j+1)*int(len(h_list0)/m_split)]))
            vs=[]
            hs=[]
            for i in range(n_split):
                v_list2=torch.stack(list(v_list1[i*int(len(v_list1)/n_split):(i+1)*int(len(v_list1)/n_split)]))
                h_list2=torch.stack(list(h_list1[i*int(len(h_list1)/n_split):(i+1)*int(len(h_list1)/n_split)]))
                if CUDA:
                    v_list2=v_list2.to(device='cuda:' + str(i)).view(len(v_list2), n_vis)
                    h_list2=h_list2.to(device='cuda:' + str(i)).view(len(h_list2), n_hid)
                else:
                    None
                vs.append(v_list2)
                hs.append(h_list2)
            for i in range(n_split): 
                if CUDA:
                    a=self.v_bias.to(device='cuda:' + str(i)).view(n_vis)
                    b=self.h_bias.to(device='cuda:' + str(i)).view(n_hid)
                    W=self.Weight.to(device='cuda:' + str(i)).view(n_hid, n_vis)
                    e=(-torch.matmul(vs[i].float(), a)-torch.diagonal(torch.matmul(torch.matmul(vs[i].float(), W.t()), hs[i].float().t()))-torch.matmul(hs[i].float(), b)).to('cuda:0')
                    e_list.append(e)
                else:
                    a=self.v_bias.view(n_vis)
                    b=self.h_bias.view(n_hid)
                    W=self.Weight.view(n_hid, n_vis)
                    e=(-torch.matmul(vs[i].float(), a)-torch.diagonal(torch.matmul(torch.matmul(vs[i].float(), W.t()), hs[i].float().t()))-torch.matmul(hs[i].float(), b))
                    e_list.append(e)
        return torch.stack(e_list).view(len(v_list0))
    
    def forward(self, v):
        h = self.v2h(v)
        h = h.bernoulli()
        for _ in range(self.k):
            v_gibbs = self.h2v(h).to(device)
            v_gibbs = v_gibbs.bernoulli()
            h = self.v2h(v_gibbs).to(device)
            h = h.bernoulli()
        return v, v_gibbs
    

# Function to make a train data loader

In [16]:
from torch.utils.data import Dataset

class CustomDataset(Dataset): 
    def __init__(self, dataset):
        data_x = dataset
        self.x_data = data_x
#         self.y_data = data_y

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
#         y = torch.FloatTensor([self.y_data[idx]])
        return x

def data_to_loader(fullconfigs):
    fulldata=CustomDataset(fullconfigs)
    full_dataset = fulldata
    full_loader = torch.utils.data.DataLoader(full_dataset, batch_size)
    return full_loader


In [17]:
def train_and_get_data(n_hid, model, lr, train_loader):
    # Train and get the new result
    rbm=RBM(n_vis, n_hid, k)
    train_op = optim.SGD(rbm.parameters(), lr, momentum=0.9)
    rbm.train()
    train_loss_list=[]
    IG_loss_list=[]
    for epoch in range(n_epochs):
        train_loss_epoch = []
        for _, (data) in enumerate(train_loader):
            data=data.to(device)
            v, v_gibbs = rbm(data.view(-1, n_vis))
            train_loss = torch.mean(rbm.Fv(v)) - torch.mean(rbm.Fv(v_gibbs))
            train_loss_epoch.append(train_loss.item())
            train_op.zero_grad()
            train_loss.backward()
            train_op.step()
        if epoch in epoch_to_save:
            train_loss_list.append(np.mean(train_loss_epoch))
            Fv=torch.dot(rbm.Fv(v_list_ising2), Pv)
            F=-torch.log(torch.sum(torch.exp(-rbm.Energy_GPU2(v_list_rbm, h_list_rbm))))
            IG_loss_list.append(float((Fv-F-S).detach().cpu().numpy()))
            print(epoch, train_loss_list[-1], IG_loss_list[-1])

        # if min_loss>train_loss_list[-1]:
        #     min_loss=train_loss_list[-1]
        #     model_to_save=rbm
        # if epoch>100 and train_loss_list[-1]>=train_loss_list[-30] and train_loss_list[-1]>=train_loss_list[-100]:
        #     model_to_save=rbm
        #     break

    return rbm.cpu().state_dict(), train_loss_list, IG_loss_list


In [23]:
# Hyper parameter들을 설정
n_vis=9
n_hid=12
k=5
n_epochs=8195
lr=0.01
batch_size=5120
epoch_to_save=[1,2,4,8,16,32,64,128,256,512,1024,2048,4096,8192]

In [24]:
torch.set_printoptions(precision=10)
for n_hid in [2,4,8,16]:
    v_list_rbm=[]; h_list_rbm=[]
    for s in tnrange(2**(n_vis+n_hid)):
        full=decimal_to_binary(s, n_hid+n_vis)[0]
        v=full[:n_vis]; h=full[-n_hid:]
        v_list_rbm.append(v); h_list_rbm.append(h)
    v_list_rbm=torch.stack(v_list_rbm).to(device)
    h_list_rbm=torch.stack(h_list_rbm).to(device)

    v_list_ising=[]
    v_list_ising2=[]
    for s in range(2**n_vis):
        v=decimal_to_binary(s, n_vis)[0]
        v_list_ising.append(np.reshape(v,(3,3))*2-1)
        v_list_ising2.append(v)
    v_list_ising2=torch.stack(v_list_ising2).to(device)        
    for T in [1.47, 1.78, 2.3, 5.2, 16]:
        fullconfigs=pd.read_pickle('{base}/loss_IG/3*3/3*3_full_T={T}.pkl'.format(base=base, T=T))
        loader_list=[]
        for i in range(10):
            random.shuffle(fullconfigs[i])
            loader_list.append(data_to_loader(fullconfigs[i]))

        bf_list=np.exp(-Ising_energy(v_list_ising)/T)
        S=entropy(bf_list)
        Pv=torch.tensor(bf_list/sum(bf_list)).to(device)
        # try:
        #     old_model=pd.read_pickle('{base}/loss_IG/3*3/state_dict/model_n_hid={n_hid}_T={T}.pkl'.format(base=base, n_hid=n_hid, T=T))
        # except:
        #     old_model=0
        # dict_model={}
        # dict_train_loss={}
        # dict_IG_loss={}
        # for m in range(10):
        model0, train_loss_list, IG_loss_list=train_and_get_data(n_hid, 0, lr=lr, train_loader=loader_list[0])
            # dict_model[str(m)]=model0
            # dict_train_loss[str(m)]=train_loss_list
            # dict_IG_loss[str(m)]=IG_loss_list
        with open('{base}/loss_IG/3*3/state_dict/model_n_hid={n_hid}_T={T}_both.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump(model0, f)
        with open('{base}/loss_IG/3*3/loss/train_loss_n_hid={n_hid}_T={T}_both.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump(np.array(train_loss_list), f)
        with open('{base}/loss_IG/3*3/loss/IG_loss_n_hid={n_hid}_T={T}_both.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump(np.array(IG_loss_list), f)


1 1.8154195308685304 6.232266426086426
2 1.0210851669311523 5.779863357543945
4 0.21613926887512208 5.244598388671875
8 -0.397713828086853 4.757726669311523
16 -1.289702868461609 3.7593166828155518
32 -2.250282049179077 1.9934885501861572
64 -1.637998390197754 2.0286476612091064
128 -0.9430793762207031 3.749110460281372


KeyboardInterrupt: ignored